In [2]:
import statsapi
import pandas as pd
from tqdm.notebook import tqdm

# -----------------------------
# Step 1: Get game list
# -----------------------------
game_list = statsapi.schedule(start_date='2025-03-28', end_date='2025-07-31', sportId=1)
game_pks = [game['game_id'] for game in game_list]
print(f"✅ Found {len(game_pks)} games.")

# -----------------------------
# Step 2: Ingest play-by-play data
# -----------------------------
all_plays = []

for gamePk in tqdm(game_pks, desc="Fetching play-by-play"):
    try:
        data = statsapi.get('game_playByPlay', {'gamePk': gamePk})
        plays = data.get('allPlays', [])
        all_plays.extend(plays)
    except Exception as e:
        print(f"Error fetching gamePk {gamePk}: {e}")

print(f"✅ Total plays ingested: {len(all_plays)}")

# -----------------------------
# Step 3: Define extraction functions
# -----------------------------
def extract_result(play, event):
    return {
        'playId': event['playId'],
        'startTime': event['startTime'],
        'endTime': event['endTime'],
        'type': play['result'].get('type'),
        'event': play['result'].get('event'),
        'eventType': play['result'].get('eventType'),
        'description': play['result'].get('description'),
        'awayScore': play['result'].get('awayScore'),
        'homeScore': play['result'].get('homeScore'),
        'isOut': play['result'].get('isOut')
    }

def extract_about(play, event):
    about = play['about']
    return {
        'playId': event['playId'],
        'startTime': event['startTime'],
        'endTime': event['endTime'],
        'atBatIndex': about.get('atBatIndex'),
        'halfInning': about.get('halfInning'),
        'isTopInning': about.get('isTopInning'),
        'inning': about.get('inning'),
        'isComplete': about.get('isComplete'),
        'isScoringPlay': about.get('isScoringPlay'),
        'hasOut': about.get('hasOut')
    }

def extract_count(play, event):
    count = play['count']
    return {
        'playId': event['playId'],
        'startTime': event['startTime'],
        'endTime': event['endTime'],
        'balls': count.get('balls'),
        'strikes': count.get('strikes'),
        'outs': count.get('outs')
    }

def extract_matchup(play, event):
    matchup = play['matchup']
    return {
        'playId': event['playId'],
        'startTime': event['startTime'],
        'endTime': event['endTime'],
        'batterId': matchup['batter'].get('id'),
        'batterName': matchup['batter'].get('fullName'),
        'batSide': matchup['batSide'].get('code'),
        'pitcherId': matchup['pitcher'].get('id'),
        'pitcherName': matchup['pitcher'].get('fullName'),
        'pitchHand': matchup['pitchHand'].get('code'),
        'menOnBase': matchup['splits'].get('menOnBase'),
        'rbi': play['result'].get('rbi'),
        'isScoringEvent': play['result'].get('isOut')
    }

def extract_pitch_detail(event):
    pd = event.get('pitchData', {})
    coords = pd.get('coordinates', {})
    breaks = pd.get('breaks', {})
    pitch_type = event.get('details', {}).get('type', {}).get('description')
    return {
        'playId': event['playId'],
        'startTime': event['startTime'],
        'endTime': event['endTime'],
        'isInPlay': event['details'].get('isInPlay'),
        'isStrike': event['details'].get('isStrike'),
        'isBall': event['details'].get('isBall'),
        'code': event['details'].get('code'),
        'description': event['details'].get('description'),
        'isOut': event['details'].get('isOut'),
        'pitchType': pitch_type,  # ← Added here
        'startSpeed': pd.get('startSpeed'),
        'strikeZoneTop': pd.get('strikeZoneTop'),
        'strikeZoneBottom': pd.get('strikeZoneBottom'),
        'aY': coords.get('aY'),
        'aZ': coords.get('aZ'),
        'pfxX': coords.get('pfxX'),
        'pfxZ': coords.get('pfxZ'),
        'pX': coords.get('pX'),
        'pZ': coords.get('pZ'),
        'vX0': coords.get('vX0'),
        'vY0': coords.get('vY0'),
        'vZ0': coords.get('vZ0'),
        'x': coords.get('x'),
        'y': coords.get('y'),
        'x0': coords.get('x0'),
        'y0': coords.get('y0'),
        'z0': coords.get('z0'),
        'aX': coords.get('aX'),
        'breakAngle': breaks.get('breakAngle'),
        'breakLength': breaks.get('breakLength'),
        'breakY': breaks.get('breakY'),
        'breakVertical': breaks.get('breakVertical'),
        'breakHorizontal': breaks.get('breakHorizontal'),
        'spinRate': breaks.get('spinRate'),
        'spinDirection': breaks.get('spinDirection'),
        'zone': pd.get('zone'),
        'pitchNumber': event.get('pitchNumber'),
        'isPitch': event.get('isPitch'),
        'type': event.get('type')
    }
def extract_hit_data(event):
    hd = event.get('hitData', {})
    coords = hd.get('coordinates', {})
    return {
        'playId': event['playId'],
        'startTime': event['startTime'],
        'endTime': event['endTime'],
        'launchSpeed': hd.get('launchSpeed'),
        'launchAngle': hd.get('launchAngle'),
        'totalDistance': hd.get('totalDistance'),
        'trajectory': hd.get('trajectory'),
        'hardness': hd.get('hardness'),
        'location': hd.get('location'),
        'coordX': coords.get('coordX'),
        'coordY': coords.get('coordY'),
        'index': event.get('index')
    }

# -----------------------------
# Step 4: Extract rows
# -----------------------------
result_rows, about_rows, count_rows = [], [], []
matchup_rows, pitch_rows, hit_rows = [], [], []

for play in all_plays:
    for event in play.get('playEvents', []):
        if event.get('isPitch'):
            result_rows.append(extract_result(play, event))
            about_rows.append(extract_about(play, event))
            count_rows.append(extract_count(play, event))
            matchup_rows.append(extract_matchup(play, event))
            pitch_rows.append(extract_pitch_detail(event))
            if 'hitData' in event:
                hit_rows.append(extract_hit_data(event))

# -----------------------------
# Step 5: Create DataFrames
# -----------------------------
df_result = pd.DataFrame(result_rows)
df_about = pd.DataFrame(about_rows)
df_count = pd.DataFrame(count_rows)
df_matchup = pd.DataFrame(matchup_rows)
df_pitch = pd.DataFrame(pitch_rows)
df_hit = pd.DataFrame(hit_rows)

# -----------------------------
# Step 6: Optional CSV export
# -----------------------------
# df_result.to_csv('df_result.csv', index=False)
# df_about.to_csv('df_about.csv', index=False)
# df_count.to_csv('df_count.csv', index=False)
# df_matchup.to_csv('df_matchup.csv', index=False)
# df_pitch.to_csv('df_pitch.csv', index=False)
# df_hit.to_csv('df_hit.csv', index=False)

✅ Found 1652 games.


Fetching play-by-play:   0%|          | 0/1652 [00:00<?, ?it/s]

✅ Total plays ingested: 124509


In [5]:
# Step 6: Optional CSV export
# -----------------------------
df_result.to_csv('df_result.csv', index=False)
df_about.to_csv('df_about.csv', index=False)
df_count.to_csv('df_count.csv', index=False)
df_matchup.to_csv('df_matchup.csv', index=False)
df_pitch.to_csv('df_pitch.csv', index=False)
df_hit.to_csv('df_hit.csv', index=False)